# Disclaimer:

I am aware that most of this model's predictions will actually be negative values and although these negative predictions for practical business purposes might not be useful, they might actually lead to a better MSE when this model is being evaluated.

For future references, I would actually model this problem differently. Instead aggregating on an entity level, I would aggerate the data on a conversion level, so that conversion would become our target feature and would always be either 0 or 1. Then, instead of making this a regression problem, I would approach it as a calibrated classification problem, with the output being the probability of conversion. Mind you, that for reliable results, the classification must be calibrated, so that the classification probabilities can be considered valid. Lastly, instead of MSE, a metric that measures probabilistic predictions, such as the Brier Score, might be adequate.

Since the instructions for this task were quite clear, I followed them regardless of my possible objections.

In [532]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor
import category_encoders as ce
from sklearn.metrics import mean_squared_error, make_scorer
from sklearn.feature_selection import SelectFromModel
from xgboost import plot_importance
from matplotlib import pyplot as plt
from sklearn.svm import LinearSVR
from sklearn.linear_model import SGDRegressor
from skopt import BayesSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import IsolationForest

import warnings

# Data Import and Analysis

In [533]:
df = pd.read_csv('train.csv')

First we import the csv file containing the training data 

In [534]:
df.describe()

,entity_id,att1,att2,att3,att4,att5,att6,att7,att8,att9,...,att154,att155,att156,att157,att158,att159,att160,week,Clicks,Conversions
count,44945.000000,44945.000000,44945.000000,44945.000000,44945.000000,44945.000000,44945.000000,44945.000000,44945.000000,44945.000000,...,44945.000000,44945.000000,44945.000000,44945.000000,44945.000000,44945.000000,44945.000000,44945.000000,44945.000000,44945.000000
mean,11357.256647,0.450973,0.349694,0.921170,0.549027,0.644210,0.349694,0.391946,0.050328,0.022939,...,0.003560,0.002403,0.003604,0.390077,0.003204,0.003538,0.003404,30.494271,8.055779,0.467727
std,6715.822919,0.497596,0.476879,0.269476,0.497596,0.478757,0.476879,0.488190,0.218624,0.149711,...,0.059559,0.048961,0.059929,0.487773,0.056513,0.059374,0.058246,0.499973,30.756230,1.963123
min,4.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,30.000000,1.000000,0.000000
25%,4789.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,30.000000,1.000000,0.000000
50%,12194.000000,0.000000,0.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,30.000000,2.000000,0.000000
75%,16640.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,31.000000,5.000000,0.000000
max,22836.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,31.000000,1801.000000,90.000000


# Outlier Detection and Removal

we next take a high level look on some of the columns. This confirms our previous observation that all the attribute columns simply binary colums containing either 0 or 1 as a value, while the columns "week" contains either the value 30 or 31. Among the attribute columns we notice that some of them are highly imbalanced towards either 0 or 1.

I also notice that the max values for the columns Conversions and Clicks are strangely high with 90 for Conversions and 1801 for Clicks. As these could be outliers, I will employ an Isolation Forest algorithm to partition the feature space and thus identify and eliminate outliers, which can possibly have a hugely positive effect on the model's performance:

In [535]:
def isolation_outlier(df, cols=None):
    print('dataset with outliers:', len(df))
    isof = IsolationForest(max_samples=1000, contamination=.075)
    if cols == None:
        isof.fit(df.select_dtypes(include=np.number))
        listpred = isof.predict(df.select_dtypes(include=np.number))
    else:
        isof.fit(df[cols])
        listpred = isof.predict(df[cols])
    df['outlier'] = listpred
    df = df.loc[df['outlier'] == 1]
    df.drop('outlier', axis=1, inplace=True)
    print('dataset without outliers:', len(df))
    return df

In [536]:
df = isolation_outlier(df, cols=['Conversions', 'Clicks'])

dataset with outliers: 44945
dataset without outliers: 41582


/opt/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


Around 3500 rows were eliminated and as expected the column mean as well as the max value for both columns has decreased.

In [537]:
df[['Conversions', 'Clicks']].describe()

,Conversions,Clicks
count,41582.000000,41582.000000
mean,0.141071,3.651099
std,0.419415,4.403579
min,0.000000,1.000000
25%,0.000000,1.000000
50%,0.000000,2.000000
75%,0.000000,4.000000
max,3.000000,30.000000


# Feature Engineering

In [538]:
df['conversion_rate'] = df['Conversions'] / df['Clicks']
df.drop(['Conversions', 'Clicks', 'entity_id'], axis=1, inplace=True)

While we're at it, let's also manually one hot encode the week feature, so that it is uniformly scaled with the attribute features, which might give us a slight boost in predictive power:

In [539]:
week_30_binary = []
for row in df['week'].iteritems():
    if row[1] == 30:
        week_30_binary.append(1)
    else:
        week_30_binary.append(0)

df.drop('week', axis=1, inplace=True)
df['is_week_30'] = week_30_binary

Quick check for NaN values:

In [540]:
df.isnull().sum().sort_values(ascending=True)

att1          0
att105        0
att106        0
att107        0
att108        0
             ..
att57         0
att58         0
att59         0
att41         0
is_week_30    0
Length: 163, dtype: int64

# Train/Test split

Operating under the assumption that our dataset is close to a perfect reprentation of business reality (which it definitely should be!), it is a valid method to stratify the sampling of our training and test datasets accordind to our target feature.

Since our target feature is continous, though, stratifying it is not as straight forward as it would be with a binary one. So we basically create 225 bins (the number 225 was heuristically chosen) in which we 'place' instances of our target feature, so that within each bin the mean value of our target feature is equal. This we can then pass on to our "stratify" parameter, when splitting the data, so that train and test dataset have an equal distribution of the target feature:

In [541]:
bins = np.linspace(0, len(df),225)
y_binned = np.digitize(df['conversion_rate'], bins)

In [542]:
X_train, X_test, y_train, y_test = train_test_split(df.drop('conversion_rate', axis=1), df['conversion_rate'], 
                                                    test_size=0.3, random_state=42, stratify=y_binned)

df_train = pd.concat([X_train, y_train], axis=1)
df_test = pd.concat([X_test, y_test], axis=1)

# One Hot Encoding

Our device column only has 3 values, so performing a simple one hot encoding will fully suffice:

In [543]:
oh = ce.OneHotEncoder(cols=['device'], drop_invariant=True, use_cat_names=True)
oh.fit(X_train, y_train)

X_train = oh.transform(X_train)
X_test = oh.transform(X_test)

# Hyperparameter Optimization, K-Fold cross validation and Model Training

Because we are dealing with relatively high dimensional data, let's try using a linear algorithm for our regression model. Given the structure of our data, this stochastic gradient descent regression algorithm should perform well and also should not take long to fit compared to a non-parametric decision tree for example. We define a parameter space as well as MSE as our metric and pass this on to Bayesian Hyperparameter tuning instance with an in-build Cross Validation:

In [544]:
params_sgd = {"loss": ['squared_loss', 'huber', 'epsilon_insensitive','squared_epsilon_insensitive'],
              "penalty": ['l1', 'l2', 'elasticnet'],
              "alpha": [0.00001,0.0001, 0.001, 0.01, 0.1, 1],
              "max_iter": [3000],
              "early_stopping": [True]
              }

sgd = SGDRegressor()
mse_scorer = make_scorer(mean_squared_error, greater_is_better=False)
bcv = BayesSearchCV(sgd, params_sgd,cv=10, n_iter=100, random_state=42, verbose=0, n_jobs=1, n_points=300, scoring=mse_scorer)

In [545]:
with warnings.catch_warnings():
    #The Bayes Optimizer Library throws some pretty lenghty FutureWarnings during fitting. We will ignore them.
    warnings.filterwarnings("ignore")
    bcv.fit(X_train, y_train)

After the hyperparameter optimization and the cross validation, we fit the the algorithm with the best parameters to the entire training dataset:

In [546]:
best = bcv.best_estimator_
best.fit(X_train, y_train)

SGDRegressor(alpha=1e-05, average=False, early_stopping=True, epsilon=0.1,
             eta0=0.01, fit_intercept=True, l1_ratio=0.15,
             learning_rate='invscaling', loss='squared_loss', max_iter=3000,
             n_iter_no_change=5, penalty='l1', power_t=0.25, random_state=None,
             shuffle=True, tol=0.001, validation_fraction=0.1, verbose=0,
             warm_start=False)

# Prediction, Baselines and Result

We then predict on our test data. Also we create baseline predictions that either contain just zeroes, the mean of the entire dataset or the respective mean for each device.

The results show that our algorithm has outperfomed the baselines. 

In [547]:
y_sgd_pred = best.predict(X_test)
y_pred_mean = np.tile(df['conversion_rate'].mean(), len(y_sgd_pred))
y_pred_zero = np.tile(np.array(0),len(y_sgd_pred))

means = df_train.groupby('device')['conversion_rate'].mean()
y_pred_device = []
for row in df_test['device'].iteritems():
    if row[1] == 'Computer':
        y_pred_device.append(means[0])
    elif row[1] == 'Smartphone':
        y_pred_device.append(means[1])
    elif row[1] == 'Tablet':
        y_pred_device.append(means[2])
    else:
        print('Error')

mse_zero = mean_squared_error(y_test, y_pred_zero)
mse_mean = mean_squared_error(y_test, y_pred_mean)
mse_device = mean_squared_error(y_test, y_pred_device)
mse_sgd = mean_squared_error(y_test, y_sgd_pred)


print('mse sgd: ', round(mse_sgd, 4))
print('mse device: ', round(mse_device, 4))
print('mse mean: ', round(mse_mean, 4))
print('mse zero: ', round(mse_zero,4))

mse sgd:  0.0219
mse device:  0.0221
mse mean:  0.0222
mse zero:  0.0237


# Extra: Additional Feature Engineering

Our Dataset is quite high dimensional, which obviously comes with it's own problems such as limiting our choice of algorithms as well as making model training more computationally expensive. 

The attributes in our dataset are not further described in what they mean or represent. We only know that they are all binary and that some of them are highly imbalanced. This has made me think - "Why not just add all attributes up within a row? Why not turn 160 columns into 1?" - At best, this would reduce noise in the data and provide a better performing model, at worst the model would at least be much less computationally expensive due to much reduced dimensionality.

So, this is what I did. I added up all attribute columns across each row into one single value, discarded the single attribute colums and retained the "week" and "device columns"

In [548]:
X_train_att = X_train.drop(['device_Computer', 'device_Smartphone','device_Tablet','is_week_30'], axis=1)
X_train_else = X_train[['device_Computer', 'device_Smartphone','device_Tablet', 'is_week_30']]
X_train_att_sum = X_train_att.sum(axis=1)

X_train_sum_full = pd.concat([X_train_att_sum, X_train_else], axis=1)
X_train_sum_full.shape


X_test_att = X_test.drop(['device_Computer', 'device_Smartphone','device_Tablet','is_week_30'], axis=1)
X_test_else = X_test[['device_Computer', 'device_Smartphone','device_Tablet', 'is_week_30']]
X_test_att_sum = X_test_att.sum(axis=1)

X_test_sum_full = pd.concat([X_test_att_sum, X_test_else], axis=1)
X_test_sum_full.shape

X_train_sum_full.rename(columns={0: "sum"}, inplace=True)
X_test_sum_full.rename(columns={0: "sum"}, inplace=True)

We have excluded a tree based algorithm due to the relatively high dimensionality of our data. Now that this is greatly reduced, why not give our good old friend XGBoost a shot? 

As previously done, we have parameter space as well as Bayesian Hyperparameter Optimization with a K-Fold Cross Validation:

In [549]:
params_xgr = {"n_estimators": [10,20,50,100],
              "max_depth": list(range(1, 11)),
              "learning_rate": [1e-3, 1e-2, 1e-1, 0.5, 1.],
              "subsample": list(np.arange(0.05, 1.01, 0.05)),
              "min_child_weight": list(range(1, 21)),
              "colsample_bytree": list(np.arange(0.1,1,0.1)),
              "reg_lambda": [1e-4, 1e-3, 1e-2, 1e-1, 0.5, 1., 5., 10., 15., 20., 25.],
              "reg_alpha": [1e-4, 1e-3, 1e-2, 1e-1, 0.5, 1., 5., 10., 15., 20., 25.]

              }
xgr = XGBRegressor()
bcvxg = BayesSearchCV(xgr, params_xgr,cv=10, n_iter=100, random_state=42, verbose=0, n_jobs=1, n_points=300, 
                      scoring=mse_scorer)

In [550]:
with warnings.catch_warnings():
    #The Bayes Optimizer Library throws some pretty lenghty FutureWarnings during fitting. We will ignore them.
    warnings.filterwarnings("ignore")
    bcvxg.fit(X_train_sum_full, y_train)

We again train on the whole training data:

In [551]:
best2 = bcvxg.best_estimator_
best2.fit(X_train_sum_full, y_train)
 
mse_xg = mean_squared_error(y_test, best2.predict(X_test_sum_full))

And we see that our XGBoost model actually performed almost as good as our SGD model, which was trained on the full data set.

In [552]:
print('mse sgd: ', round(mse_sgd, 4))
print('mse xg summed attributes: ', round(mse_xg, 4))
print('mse device: ', round(mse_device, 4))
print('mse mean: ',round(mse_mean, 4))
print('mse zero: ', round(mse_zero, 4))

mse sgd:  0.0219
mse xg summed attributes:  0.0221
mse device:  0.0221
mse mean:  0.0222
mse zero:  0.0237


# Validation Data Set and Export of Validation Data Set

In [553]:
val = pd.read_csv('test_w_weeks.csv')
val_trans = val.drop('entity_id', axis=1)

In [554]:
val_trans = oh.transform(val_trans)

In [555]:
val_predictions_sgd_full_data = best.predict(val_trans)
val['predictions'] = val_predictions_sgd_full_data
val_final = val[['entity_id', 'device', 'predictions']].sort_values(by=['entity_id', 'device'])

In [556]:
val_final.to_csv('results.csv', index=False)

# Achievements

In this Notebook, I was able to:

- Create a ML model that outperformed Naive baselines
- Transform data with the benefit of reducing computional cost and changing the data structure to make the data more widely usable.